In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np


Bad key "ytick.alignment" on line 268 in
/usr/share/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution

Bad key "xtick.alignment" on line 250 in
/usr/share/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from collections import Counter
from imblearn.datasets import fetch_datasets
from sklearn import svm
from sklearn.model_selection import train_test_split, KFold
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

In [3]:
df = pickle.load(open('dataframe.pickle', 'rb'))
print(df.dtypes)
df.head()

issuercountrycode                       object
txvariantcode                           object
bin                                    float64
amount                                 float64
currencycode                            object
shoppercountrycode                      object
shopperinteraction                      object
cardverificationcodesupplied            object
cvcresponsecode                          int64
creationdate                    datetime64[ns]
accountcode                             object
mail_id                                  int64
ip_id                                    int64
card_id                                  int64
label                                    int64
creationdate_timestamp                 float64
dtype: object


,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp
0,MX,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,True,0,2015-07-01,MexicoAccount,68370,111778,184798,1,1.435785e+09
59886,GB,visadebit,465858.0,3995.0,GBP,GB,Ecommerce,True,1,2015-07-01,UKAccount,308365,74702,209050,0,1.435772e+09
59887,GB,visadebit,446291.0,1895.0,GBP,GB,Ecommerce,True,1,2015-07-01,UKAccount,274879,269031,22055,0,1.435772e+09
59888,GB,visadebit,476248.0,3695.0,GBP,GB,Ecommerce,True,1,2015-07-01,UKAccount,84503,315241,211339,0,1.435772e+09
59889,GB,visadebit,446238.0,9495.0,GBP,GB,Ecommerce,True,1,2015-07-01,UKAccount,56386,48485,141732,0,1.435772e+09


In [4]:
# TODO: to verify, does this dataframe takes into account Refused transactions?
# How many entries in total ?
df.shape[0]

236698

In [5]:
# How many of those are fraudolent and how many are safe ?
s = df.label.value_counts()
print s
print s[1]/float(s[0])

0    236353
1       345
Name: label, dtype: int64
0.00145968107026


In [6]:
# TODO clean and filter only the interesting features, this is just an example:
df_interesting = df[['label','ip_id','issuercountrycode']]
df_interesting.head()

,label,ip_id,issuercountrycode
0,1,111778,MX
59886,0,74702,GB
59887,0,269031,GB
59888,0,315241,GB
59889,0,48485,GB


In [7]:
# Convert categorical variable into dummy/indicator variables for learning
df_encoded = pd.get_dummies(df_interesting)
df_encoded = df_encoded.sample(frac=1).reset_index(drop=True)

df_encoded.head()

,label,ip_id,issuercountrycode_AE,issuercountrycode_AI,issuercountrycode_AL,issuercountrycode_AM,issuercountrycode_AO,issuercountrycode_AR,issuercountrycode_AT,issuercountrycode_AU,...,issuercountrycode_TZ,issuercountrycode_UA,issuercountrycode_US,issuercountrycode_UY,issuercountrycode_VG,issuercountrycode_VN,issuercountrycode_ZA,issuercountrycode_ZM,issuercountrycode_ZW,issuercountrycode_ZZ
0,0,213614,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,227600,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,117312,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,93613,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,186406,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# # Creating training and test sets
# training_features, test_features, \
# training_target, test_target, = train_test_split(df_encoded.drop(['label'], axis=1),
#                                                df_encoded['label'],
#                                                test_size = .1,
#                                                random_state=12)
X = np.array(df_encoded.drop(['label'], axis=1))
Y = np.array(df_encoded['label'])
#print X.shape
#print Y.shape

sm = SMOTE(random_state=12)
clf = svm.SVC()

kf = KFold(n_splits=10) #Already shuffled
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    X_train, Y_train = sm.fit_sample(X_train, Y_train)

    #print("X train SIZE: " + str(len(X_train)) + " Y train size: " + str(len(Y_train)))
    #print("X test SIZE: " + str(len(X_test)) + " Y test size: " + str(len(Y_test)))
    clf.fit(X_train, Y_train)
    print(clf.score(X_test, Y_test))

('TRAIN:', array([ 23670,  23671,  23672, ..., 236695, 236696, 236697]), 'TEST:', array([    0,     1,     2, ..., 23667, 23668, 23669]))


In [ ]:
# print(training_features.shape)
# print(training_target.shape)

In [ ]:
# # SMOTE analysis (note, this is done after sampling)
# sm = SMOTE(random_state=12)
# x_res, y_res = sm.fit_sample(training_features, training_target)
# print training_target.value_counts()
# print np.bincount(y_res)
